# Weekend Movie Trip
## General Information
1. Author: Patrick McNamee
2. Date: 10/5/2019


## Description
Trying to cluster movies for recommendations. Data used is a small version of the latest dataset from https://grouplens.org/datasets/movielens/

## Initial Data Exploration
Alright, first step is to load the data into panda dataframes to observe what the data looks like in its original form. There are two sets of note which is that the movies are seperate from the users and user ratings.

In [1]:
import pandas as pd

movies = pd.read_csv('./data/original-ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [2]:
ratings = pd.read_csv('./data/original-ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


## Data Feature Engineering
So the movie dataset genres has several unique attributes with movies having one or more genres. This is a prime candadite for one-hot-encoding. Additionally we could likely extract the year of release of the movie 

In [3]:
movies['genres'] = movies['genres'].str.split('|')#Deliminate the list
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [4]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
movies = movies.join(pd.DataFrame(mlb.fit_transform(movies.pop('genres')),
                                  columns=mlb.classes_,
                                  index = movies.index)
                    )
movies.head()

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


# Clustering
## K-Means

Another thing to look at is the relative distributions of of ratings to see if there is any distinct clusters.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

movie_rating = ratings.groupby('movieId')['rating'].mean()
ax = sns.kdeplot(movie_rating, shade=True, color='blue')
plt.show()

It seems to meet expectations that there is clustering around ratings of 0.5, 1, ..., 4.5, and 5. So perhaps the quickest and easy way to cluster is simply by rating. If a user likes to watch movies that are globally around 3.5, then the algorithm should suggest other movies that are rated around 3.5.

In [ ]:
import numpy as np
from sklearn.cluster import KMeans

n_cluster_used = 10 # 0.5:5 intervaled at 0.5
kmeans = KMeans(n_clusters=n_cluster_used, random_state=0).fit(np.array(movie_rating).reshape(-1,1))
kmeans.cluster_centers_

## Weighted Graph

If we think of the users as traversing a complete graph between the movies than we can weight the edges of the graph by commonality of what users watched. This should give locally densed areas around genres as well as quality. If people who movie A always watch movie B sometime afterwards then it would make sense to recommend movie B to someone who had yet to see if if there is a strong edge. This analysis does assume that people have told each other about movies and weaker movies are seen by a lot less people.

In [ ]:
import igraph as ig

#Create the graph with movies as the verticies
g = ig.Graph.Full(len(ratings.movieId.unique()))
ig.summary(g)

In [ ]:
#Setting up graph attritubutes so that you can see the title
g.vs["name"] = movies["title"]
#movie ID is 1 indexed but vertices are 0 indexed
g.es["weight"] = 0.0 #zero weights on tree
g.is_weighted()

In [ ]:
#plotting
ig.plot(g, layout=g.layout("kk"))

In [ ]:
ig.plot(g, layout="lgl")